In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Modelos de Ensamble

## Introducción

Usando el dataset Movie_classification, que vimos en la clase de CART, vamos a 

* Entrenar tres modelos base de clasificación, y construir a partir de ellos un ensamble para predecir el valor de la variable Start_Tech_Oscar usando predict_proba y AUC como métrica

* Comparar la performance de los modelos base con la del modelo de ensamble, y con el resultado del checkpoint.


## Dataset

En esta clase usaremos un dataset con info de películas ("Movie_classification.csv").  
Este dataset esta conformado por los siguientes features:  

 *   **Marketing expense:**    (float64)    Gasto total en Marketing      
 *   **Production expense:**   (float64)    Gasto total de Producción
 *   **Multiplex coverage:**   (float64)    Cobertura promedio de Multiplex
 *   **Budget:**               (float64)    Presupuesto
 *   **Movie_length:**         (float64)    Duración de la película
 *   **Lead_ Actor_Rating:**   (float64)    Puntaje sobre el actor principal
 *   **Lead_Actress_rating:**  (float64)    Puntaje sobre la actriz principal
 *   **Director_rating:**      (float64)    Puntaje sobre el Director
 *   **Producer_rating:**      (float64)    Puntaje sobre el Productor
 *   **Critic_rating:**        (float64)    Puntaje que le puso la crítica
 *   **Trailer_views:**        (int64)      Cantidad de vistas del Trailer
 *   **3D_available:**         (object)     Si esta disponible en 3D (Yes/No)
 *   **Time_taken:**           (float64)    Duración de la película
 *   **Twitter_hastags:**      (float64)    Cantidad de menciones en twitter
 *   **Genre:**                (object)     Genero de la película
 *   **Avg_age_actors:**       (int64)      Edad promedio de los actores
 *   **Num_multiplex:**        (int64)      Cantidad de Multiplex
 *   **Collection:**           (int64)      Recaudación
 *   **Start_Tech_Oscar:**     (int64)      Si recibió un oscar o no.
 

## Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

## Ejercicio 1

Leamos el dataset "Movie_classification", eliminemos los registros con valores faltantes y las features categóricas "Genre" y "3D_available"

In [ ]:
data_raw = pd.read_csv("../Data/Movie_classification.csv")
print(data_raw.shape)

In [ ]:
data_no_missing = data_raw.dropna()
print(data_no_missing.shape)

In [ ]:
data = data_no_missing.drop(["Genre", "3D_available"], axis = 1)
print(data.shape)

In [ ]:
data.head()

## Ejercicio 2

Crear los conjuntos de train y test

In [ ]:
X = data.drop("Start_Tech_Oscar", axis = 1)
print(X.shape)

y = data.Start_Tech_Oscar
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 127)


## Ejercicio 3

Estandarizar las features para los conjuntos de train y test

In [ ]:
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)

X_test_scl = scaler.transform(X_test)

## Ejercicio 4

Entrenar un modelo Naive Bayes para predecir el valor de "Start_Tech_Oscar".

Evaluar su perfomance en test mediante AUC.


In [ ]:
model_1 = GaussianNB()
fit_1 = model_1.fit(X_train_scl, y_train)

In [ ]:
predict_1 = fit_1.predict_proba(X_test_scl)
predict_1_score = predict_1[:, 1]
auc_1 = roc_auc_score(y_test, predict_1_score)
print(auc_1)

## Ejercicio 5

Entrenar un modelo KNN para predecir el valor de "Start_Tech_Oscar".

Evaluar su perfomance en test mediante AUC.


In [ ]:
model_2 = KNeighborsClassifier()
fit_2 = model_2.fit(X_train_scl, y_train)

In [ ]:
predict_2 = fit_2.predict_proba(X_test_scl)
predict_2_score = predict_2[:, 1]

auc_2 = roc_auc_score(y_test, predict_2_score)
print(auc_2)

## Ejercicio 6

Entrenar unárbol de clasificación para predecir el valor de "Start_Tech_Oscar".

Evaluar su perfomance en test mediante la matriz de confusión.


In [ ]:
model_3 = tree.DecisionTreeClassifier(criterion='entropy')
fit_3 = model_3.fit(X_train_scl, y_train)

In [ ]:
predict_3 = fit_3.predict_proba(X_test_scl)
predict_3_score = predict_3[:, 1]

auc_3 = roc_auc_score(y_test, predict_3_score)
print(auc_3)

## Ejercicio 7


Construir un modelo de ensamble usando como modelos base los tres modelos anteriores.

Para esto, escribir la función `predict_proba_ensamble` que calcule la probabilidad de 1 como el promedio de las probabilidades respuesta de los predictores base

Evaluar la performance del ensamble (usando como umbral 0.5) mediante

* AUC

* accuracy

* confusion_matrix


Comparar estos resultados con los obtenidos en el ejercicio 7 del checkpoint.


In [ ]:
def predict_proba_ensamble(X, model_1, model_2, model_3):
    y_pred_1 = model_1.predict_proba(X)[:, 1]
    y_pred_2 = model_2.predict_proba(X)[:, 1]
    y_pred_3 = model_3.predict_proba(X)[:, 1]
    result = (y_pred_1 + y_pred_2 + y_pred_3) / 3    
    return result

In [ ]:
predict_ensemble = predict_proba_ensamble(X_test_scl, fit_1, fit_2, fit_3)
auc_ensemble = roc_auc_score(y_test, predict_ensemble)
print(auc_ensemble)

El valor de AUC del ensamble es mejor que el de cada uno de los modelos base.

In [ ]:
threshold = 0.5
predict_ensemble_cat = [1 if (x >= threshold) else 0 for x in predict_ensemble]  

accuracy_ensemble = accuracy_score(y_test, predict_ensemble_cat)
print(accuracy_ensemble)

confusion_matrix_ensemble = confusion_matrix(y_test, predict_ensemble_cat)
print(confusion_matrix_ensemble)

El valor de accuracy resultó similar en el ensamble construido usando el promedio de las probabilidades en lugar de la moda de las categorías

## Opcional:

¿Pueden mejorar el valor de accuracy del ensamble usando otro valor (distinto de 0.5) para el umbral? 

¿Qué pasa con la matriz de confusión cuando mejora el valor de accuracy?

Pueden hacer la prueba en un ciclo for con distintos valores de threshold, o usar roc_curve (https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html) para intentar responder.